In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession


In [4]:
spark = (
    SparkSession.builder\
    .appName("Spark")\
    .master("local")\
    .getOrCreate()
)

In [14]:
df = spark.read.csv("trips_data.csv", header=True, inferSchema=True)

In [9]:
df.count()

212

In [15]:
df =df.drop(col("Dropoff Lat"),col("Dropoff Lng"), col("Begin Trip Lat"),col("Begin Trip Lng") )

In [20]:
df = df.withColumnRenamed("City","Cidade").withColumnRenamed("Product Type","Produto").withColumnRenamed("Trip or Order Status","Status_corrida").withColumnRenamed("Request Time","Request Time")\
    .withColumnRenamed("Begin Trip Time","Inicio_corrida").withColumnRenamed("Begin Trip Address","Origem").withColumnRenamed("Dropoff Time","Hora_finalizada").withColumnRenamed("Dropoff Address","Destino")\
    .withColumnRenamed("Distance (miles)","Distancia_milhas").withColumnRenamed("Fare Amount","Preco").withColumnRenamed("Fare Currency","Moeda")

In [31]:
df =df.withColumn("Preco", col("Preco").cast(FloatType()))

In [32]:
df.show(truncate=False)

+---------+-------+--------------+-----------------------------+-----------------------------+----------------------------------------------------------------------------------+-----------------------------+--------------------------------------------------------------------------------------+----------------+-----+-----+
|Cidade   |Produto|Status_corrida|Request Time                 |Inicio_corrida               |Origem                                                                            |Hora_finalizada              |Destino                                                                               |Distancia_milhas|Preco|Moeda|
+---------+-------+--------------+-----------------------------+-----------------------------+----------------------------------------------------------------------------------+-----------------------------+--------------------------------------------------------------------------------------+----------------+-----+-----+
|Sao Paulo|UberX  |COMPLETED

In [36]:
df.groupBy("Cidade").agg(count("*").alias("Quantidade_cidade")).show(truncate=False)

+--------------+-----------------+
|Cidade        |Quantidade_cidade|
+--------------+-----------------+
|Santos        |2                |
|Sao Paulo     |178              |
|Brasilia      |2                |
|Rio de Janeiro|30               |
+--------------+-----------------+



In [51]:
df.filter(col('Cidade')=="Rio de Janeiro").orderBy("Request Time").show(30, truncate=False)

+--------------+-------+--------------+-----------------------------+-----------------------------+----------------------------------------------------------------------------------------------------------------------+-----------------------------+------------------------------------------------------------------------------------------------------------+----------------+-----+-----+
|Cidade        |Produto|Status_corrida|Request Time                 |Inicio_corrida               |Origem                                                                                                                |Hora_finalizada              |Destino                                                                                                     |Distancia_milhas|Preco|Moeda|
+--------------+-------+--------------+-----------------------------+-----------------------------+----------------------------------------------------------------------------------------------------------------------+--------

In [53]:
df.filter(col('Cidade')=="Brasilia").orderBy("Request Time").show(30, truncate=False)

+--------+-----------+--------------+-----------------------------+-----------------------------+-------------------------------------------------------------------------+-----------------------------+-------------------------------------------------------------------------+----------------+-----+-----+
|Cidade  |Produto    |Status_corrida|Request Time                 |Inicio_corrida               |Origem                                                                   |Hora_finalizada              |Destino                                                                  |Distancia_milhas|Preco|Moeda|
+--------+-----------+--------------+-----------------------------+-----------------------------+-------------------------------------------------------------------------+-----------------------------+-------------------------------------------------------------------------+----------------+-----+-----+
|Brasilia|UberX      |COMPLETED     |2021-01-30 19:05:47 +0000 UTC|2021-01-30 19:08:0

In [54]:
df_cancel = df.filter(col("Status_corrida")!="COMPLETED")


+------+-------+--------------+------------+--------------+------+---------------+-------+----------------+-----+-----+
|Cidade|Produto|Status_corrida|Request Time|Inicio_corrida|Origem|Hora_finalizada|Destino|Distancia_milhas|Preco|Moeda|
+------+-------+--------------+------------+--------------+------+---------------+-------+----------------+-----+-----+
+------+-------+--------------+------------+--------------+------+---------------+-------+----------------+-----+-----+



In [50]:
df = df.filter(col("Status_corrida")=="COMPLETED")

In [58]:
df.printSchema()

root
 |-- Cidade: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Status_corrida: string (nullable = true)
 |-- Request Time: string (nullable = true)
 |-- Inicio_corrida: string (nullable = true)
 |-- Origem: string (nullable = true)
 |-- Hora_finalizada: string (nullable = true)
 |-- Destino: string (nullable = true)
 |-- Distancia_milhas: double (nullable = true)
 |-- Preco: float (nullable = true)
 |-- Moeda: string (nullable = true)



In [77]:
gasto_total = df.agg({'Preco': 'sum'})
gasto_total = gasto_total.withColumn("sum(Preco)", format_number(round(col("sum(Preco)"),2),2))
gasto_total.show()

+----------+
|sum(Preco)|
+----------+
|  4,891.72|
+----------+



In [80]:
df.agg({'Preco': 'max'}).show()

+----------+
|max(Preco)|
+----------+
|     93.05|
+----------+



In [83]:
df.filter(col("Preco")>93.04).show(truncate=False)

+---------+-------+--------------+-----------------------------+-----------------------------+---------------------------------------------------------------------------------------+-----------------------------+-----------------------------------------------------------------------------------+----------------+-----+-----+
|Cidade   |Produto|Status_corrida|Request Time                 |Inicio_corrida               |Origem                                                                                 |Hora_finalizada              |Destino                                                                            |Distancia_milhas|Preco|Moeda|
+---------+-------+--------------+-----------------------------+-----------------------------+---------------------------------------------------------------------------------------+-----------------------------+-----------------------------------------------------------------------------------+----------------+-----+-----+
|Sao Paulo|UberX  |COM

In [89]:
df.orderBy("Distancia_milhas").show(truncate=False)

+--------------+-------+--------------+-----------------------------+-----------------------------+----------------------------------------------------------------------------------------------------+-----------------------------+------------------------------------------------------------------------------------------------------------+----------------+-----+-----+
|Cidade        |Produto|Status_corrida|Request Time                 |Inicio_corrida               |Origem                                                                                              |Hora_finalizada              |Destino                                                                                                     |Distancia_milhas|Preco|Moeda|
+--------------+-------+--------------+-----------------------------+-----------------------------+----------------------------------------------------------------------------------------------------+-----------------------------+--------------------------------

In [86]:
df.agg({'Distancia_milhas': 'max'}).show()

+---------------------+
|max(Distancia_milhas)|
+---------------------+
|                 31.8|
+---------------------+



In [90]:
df.filter(col('Distancia_milhas')>31.7).show()

+---------+-------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-----+-----+
|   Cidade|Produto|Status_corrida|        Request Time|      Inicio_corrida|              Origem|     Hora_finalizada|             Destino|Distancia_milhas|Preco|Moeda|
+---------+-------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-----+-----+
|Sao Paulo|  UberX|     COMPLETED|2018-11-16 17:47:...|2018-11-16 17:49:...|Praça Comandante ...|2018-11-16 18:54:...|R. Lourenço Leite...|            31.8|83.87|  BRL|
+---------+-------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-----+-----+



In [87]:
df.agg({'Distancia_milhas': 'min'}).show()

+---------------------+
|min(Distancia_milhas)|
+---------------------+
|                 0.01|
+---------------------+

